In [1]:
import os

In [2]:
base_dir = "./experiment/kitti/"
dirname_list = ["tulip_base_mamba_old", "tulip_base", "tulip_base_mamba", "tulip_base_mamba_crsatten", "tulip_base_mamba_crsmamba"]
result_map = {
    dirname: os.path.join(base_dir, dirname, "results_mcdrop.txt") for dirname in dirname_list
}
print(result_map)

{'tulip_base_mamba_old': './experiment/kitti/tulip_base_mamba_old/results_mcdrop.txt', 'tulip_base': './experiment/kitti/tulip_base/results_mcdrop.txt', 'tulip_base_mamba': './experiment/kitti/tulip_base_mamba/results_mcdrop.txt', 'tulip_base_mamba_crsatten': './experiment/kitti/tulip_base_mamba_crsatten/results_mcdrop.txt', 'tulip_base_mamba_crsmamba': './experiment/kitti/tulip_base_mamba_crsmamba/results_mcdrop.txt'}


In [3]:
import json
def read_json(path):
    with open(path, "r") as f:
        obj = json.load(f)
    return obj

# Example

In [4]:
# example
obj_exp = read_json(result_map["tulip_base_mamba_crsatten"])
print(obj_exp.keys())

dict_keys(['mae', 'chamfer_dist', 'iou', 'precision', 'recall', 'f1'])


In [5]:
from typing import Dict, List, Any, Tuple
import numpy as np
import pandas as pd

def make_dataframes(scores: Dict[str, List[float]], k: int = 10) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    scores: dict like {'mae': [...], 'chamfer_dist': [...], 'iou': [...], 'precision': [...], 'recall': [...], 'f1': [...]}
            Each list must have the same length.
    k: how many lowest/highest to collect per metric.
    Returns: (data_df, sum_df)
    """
    # 1) Dict -> DataFrame (each key = column)
    data_df: pd.DataFrame = pd.DataFrame(scores)

    # 2) Summary DF: per-metric mean, lowest k, highest k (values + indices)
    summary_rows: dict[str, dict[str, Any]] = {}
    n: int = len(data_df)
    kk: int = min(k, n)

    for col in data_df.columns:
        arr: np.ndarray = data_df[col].to_numpy(dtype=float)
        order: np.ndarray = np.argsort(arr)  # ascending

        low_idx: np.ndarray = order[:kk]
        high_idx: np.ndarray = order[-kk:][::-1]
        if col=="mae": arr *= 64
        summary_rows[col] = {
            "mean": float(arr.mean()), # multiply by 64
            "lowest10_indices": low_idx.tolist(),
            "lowest10_values": arr[low_idx].tolist(),
            "highest10_indices": high_idx.tolist(),
            "highest10_values": arr[high_idx].tolist(),
        }

    sum_df: pd.DataFrame = pd.DataFrame.from_dict(summary_rows, orient="index")
    sum_df.index.name = "metric"

    return data_df, sum_df

In [6]:
data_df, sum_df = make_dataframes(obj_exp)

In [7]:
data_df.head(10)

,mae,chamfer_dist,iou,precision,recall,f1
0,0.765229,0.228817,0.261497,0.406008,0.423526,0.414582
1,0.506394,0.139606,0.329256,0.479044,0.512910,0.495399
2,0.824973,0.345960,0.264713,0.409484,0.428159,0.418613
3,0.116605,0.047754,0.410906,0.575796,0.589303,0.582471
4,0.094148,0.037396,0.426667,0.591666,0.604738,0.598131
5,1.363131,0.536594,0.292470,0.445331,0.460060,0.452575
6,0.114208,0.030235,0.409243,0.575203,0.586503,0.580798
7,0.128394,0.079967,0.363244,0.530629,0.535214,0.532912
8,0.182021,0.085170,0.330799,0.491513,0.502904,0.497143
9,0.111999,0.076770,0.406989,0.574126,0.582991,0.578525


In [8]:
sum_df

,mean,lowest10_indices,lowest10_values,highest10_indices,highest10_values
metric,,,,,
mae,0.363865,"[19, 244, 89, 44, 117, 173, 362, 46, 129, 453]","[0.06830423325300217, 0.0701000839471817, 0.07...","[410, 307, 240, 407, 429, 325, 122, 71, 114, 470]","[1.7482010126113892, 1.6385953426361084, 1.604..."
chamfer_dist,0.142618,"[46, 305, 314, 351, 19, 412, 10, 251, 347, 44]","[0.01451134867966175, 0.014550074934959412, 0....","[114, 71, 429, 410, 195, 112, 276, 507, 446, 470]","[0.7209945917129517, 0.7034013271331787, 0.690..."
iou,0.364800,"[204, 415, 407, 13, 65, 41, 282, 344, 240, 230]","[0.1952363398050299, 0.22767932489451476, 0.23...","[44, 46, 305, 431, 412, 10, 351, 395, 220, 244]","[0.4888856916768578, 0.4771102770672468, 0.463..."
precision,0.523124,"[204, 415, 407, 287, 13, 143, 403, 316, 65, 463]","[0.2965532259068002, 0.34826883910386963, 0.36...","[44, 46, 431, 244, 220, 395, 305, 351, 512, 339]","[0.6486589672616206, 0.6418601659484144, 0.627..."
recall,0.540011,"[204, 407, 65, 282, 41, 344, 13, 415, 14, 240]","[0.36364676711806215, 0.385850837503371, 0.390...","[44, 10, 412, 46, 246, 138, 305, 351, 431, 419]","[0.6649706457925636, 0.6556133421968482, 0.655..."
f1,0.531313,"[204, 415, 407, 13, 65, 41, 282, 344, 240, 230]","[0.3266907695207416, 0.3709100907341215, 0.375...","[44, 46, 305, 431, 412, 10, 351, 395, 220, 244]","[0.656713533362323, 0.6460049523377946, 0.6334..."


# Run and Save

In [ ]:
for dirname, path in result_map.items():
    obj = read_json(path)
    data_df, sum_df = make_dataframes(obj)
    data_df.to_csv(os.path.join(base_dir, f"{dirname}_data.csv"), index = False, enoding="utf-8-sig")
    sum_df.to_csv(os.path.join(base_dir, f"{dirname}_sum.csv"), index = False, enoding="utf-8-sig")